In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('BFS-Superhero').getOrCreate()
sc = spark.sparkContext

In [2]:
# The characters we wish to find the degree of separation between:
startCharacterID = 5306 #SpiderMan
targetCharacterID = 14  #ADAM 3,031 (who?)

hitsCount = sc.accumulator(0)
nodesEvaluated = sc.accumulator(0)
exploredNodes = sc.accumulator(0)
createdNodes = sc.accumulator(0)

In [3]:
# COLOR: 1 - WHITE, 2 - GRAY, 3 - BLACK
def node_connections_parse (line):
    nodes = line.split()
    hero = int(nodes[0])
    connections = []
    for node in nodes[1:]:
        connections.append(int(node))

    color = 1
    distance = 9999
    if hero == startCharacterID:
        color = 2
        distance = 0

    return (hero, (connections, distance, color))

In [4]:
def merge_connections (tuple1, tuple2):
    """
    Agrupa conexões, no arquivo marvel graph podem existir diferentes linhas para cada hero
    """
    connections = tuple1[0]
    connections.extend(tuple2[0])
    return (connections, tuple1[1], tuple1[2])

In [5]:
graph = sc.textFile('file:////Users/giovanna/Documents/GitHub/pyspark/SparkCourse/Marvel-graph.txt')
graph = graph.map(node_connections_parse) # total size do not change!
graph = graph.reduceByKey(lambda x,y: merge_connections(x,y))

In [6]:
def bfsMap (node):
    """
    Para o registro avaliado identifica se a cor é cinza (cor de exploração),
        caso positivo cria para cada uma das conexoes um novo nó com a distancia
        nova e a cor de exploracao.
        Atualiza o registro atual com a nova cor (de já processado = BLACK).
    """
    nodesEvaluated.add(1)
    heroId = node[0]
    data = node[1]
    connections = data[0]
    distance = data[1]
    color = data[2]

    newNodes = []

    # GRAY mean to explore
    if color == 2:
        exploredNodes.add(1)
        # criar novas conexoes
        for connection in connections:
            createdNodes.add(1)
            newNodes.append((connection, ([], distance + 1, 2))) # (ID, (CONNECTIONS, DISTANCE, COLOR=GRAY))
            if connection == targetCharacterID:
                hitsCount.add(1) # soma o acumulador
        color = 3
    newNodes.append((heroId, (connections, distance, color))) # altera cor BLACK para o registro avaliado
    return newNodes

In [7]:
def clean_rdd (tuple1, tuple2):
    """
    Identifica códigos repetidos e mantém os dados de interesse.
    Menor distancia, maior coloração e conexões.
    """
    connections = tuple1[0] if len(tuple1[0]) > 0 else tuple2[0]
    distance = tuple1[1] if tuple1[1] < tuple2[1] else tuple2[1]
    color = tuple1[2] if tuple1[2] > tuple2[2] else tuple2[2]
    return (connections, distance, color) # não necessita retornar chave, pois somente valores são passados

In [8]:
for it in range(0,10): # busca por até 10 saltos
    total = graph.count()
    print (f"Iteração {it} total de nós {total}.")
    # processa nós de cor cinza
    expanded_graph = graph.flatMap(bfsMap)
    print (f"Total de nós gerados {expanded_graph.count()-total} após {nodesEvaluated.value} nós avaliados,  {exploredNodes.value} nós explorados e {createdNodes} nós criados.") # forca execução do flatMap para identificar se já encontrou o alvo

    if hitsCount.value > 1:
        print (f"Alvo encontrado com distância de {it}, por {hitsCount.value} caminhos.")
        break

    # limpa RDD, para que não se tenham chaves duplicadas
    graph = expanded_graph.reduceByKey(clean_rdd)

Iteração 0 total de nós 6486.
Total de nós gerados 1741 após 6486 nós avaliados,  1 nós explorados e 1741 nós criados.
Iteração 1 total de nós 6486.
Total de nós gerados 214129 após 19458 nós avaliados,  1743 nós explorados e 217611 nós criados.
Iteração 2 total de nós 6486.
Total de nós gerados 120261 após 32430 nós avaliados,  8140 nós explorados e 552001 nós criados.
Alvo encontrado com distância de 2, por 16 caminhos.


In [10]:
spark.stop()